Nepal Earthquake OSM Data Exploration
=====================================

In [1]:
require '/opt/epic-osm/epic-osm.rb'
analysis_window = './nepal-earthquake.yml'

"./nepal-earthquake.yml"

In [7]:
o  = EpicOSM.new(analysis_window: analysis_window)
aw = o.analysis_window; nil

Successfully initialized Analysis Window: Nepal Earthquake



In [ ]:
importer = 